In [1]:
import pathlib
import os
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import tensorflow as tf
from numpy.random import seed
import numpy as np
import jieba
import time
from keras.callbacks import ModelCheckpoint
import re
from sklearn.model_selection import train_test_split



seed(1)
# tf.random.set_seed(2)
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.《》（）+-=()""''/="

# skipped directories
# SKIP = ["0", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
SKIP = ["0", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
# SKIP = []

t1 = time.time()

def get_all_items(root: pathlib.Path, exclude):
    itemList = []
    for item in root.iterdir():
        if item.name in exclude:
            continue
        if item.is_dir():
            itemList.append(get_all_items(item, []))
            continue
        itemList.append(item)
    return itemList


# begin preprocessing
largeDir = pathlib.Path("./Books")
# largeDir = pathlib.Path("./Books")
BookList = get_all_items(largeDir, SKIP)
BookList = [item for sublist in BookList for item in sublist]


# clean the dataset
# for path in BookList:
#     print(path)
#     file = open(path, 'r')
#     try:
#         fileStr = file.read()
#     except UnicodeDecodeError as error:
#         file.close()
#         os.remove(path)
#     continue

bigString = ""

for path in BookList:
    with open(path, 'r', encoding='gbk') as fiction:
        bigString += fiction.read()

# methods to strip punctuation and symbols
# bigString = re.sub(r"[%s]+" %punc, "", bigString)
bigString = re.sub(r'[^\w\s]', '', bigString)
cleaned = re.sub(r'_','', bigString)
cleaned1 = re.sub(r'(\n+)', '', cleaned)

# list of the words in their original order
# allTokens = jieba.lcut(cleaned1, cut_all=False)
allTokens = [*cleaned1]
t2 = time.time()
print("Runtime for this cell in seconds: ", t2 - t1)
print("Corpus length in words: ", len(allTokens))

Runtime for this cell in seconds:  6.048093795776367
Corpus length in words:  23548408


In [11]:
minFreq = 1023
maxFreq = 50000000
wordFreq = {}
for token in allTokens:
    wordFreq[token] = wordFreq.get(token, 0) + 1

skipWords = set()
u = 0
for k, v in wordFreq.items():
    # if 1000 > wordFreq[k] > 0:
    #     u += wordFreq[k]
    if wordFreq[k] < minFreq or wordFreq[k] > maxFreq:
        skipWords.add(k)
    elif k.isascii():
        skipWords.add(k)

print(u)
# skipWords.remove("\n")
skipWords.remove(" ")
skipWords.add("\u3000")
words = set(allTokens)
print("Unique words before filter: ", len(words))
print("To reduce vocab size, neglect words with appearances < ", minFreq)
print("To reduce vocab size, neglect words with appearances > ", maxFreq)
words = sorted(set(words) - skipWords)
print("Unique words after filter: ", len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))
class_weights = dict((word_indices[word],1/wordFreq[word]) for word in words)

0
Unique words before filter:  5146
To reduce vocab size, neglect words with appearances <  1023
To reduce vocab size, neglect words with appearances >  50000000
Unique words after filter:  1687


In [21]:
import codecs

words_file_path = "vocab.txt"

words_file = codecs.open(words_file_path, 'w', encoding='gbk')
# hugeStr = "".join(str(words))
# words_file.write(hugeStr)

for w in words:
    if w != "\n":
        words_file.write(w)
        words_file.write("\n")

words_file.close()

print(words)

[' ', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '专', '且', '世', '业', '东', '丝', '丢', '两', '严', '个', '丫', '中', '丰', '临', '丹', '为', '主', '丽', '举', '乃', '久', '么', '义', '之', '乌', '乎', '乐', '乔', '乖', '乘', '九', '也', '习', '书', '买', '乱', '了', '争', '事', '二', '于', '亏', '云', '互', '五', '亚', '些', '亡', '交', '产', '享', '京', '亮', '亲', '人', '亿', '什', '仅', '仇', '今', '介', '仍', '从', '仔', '他', '付', '仙', '代', '令', '以', '仪', '们', '仰', '件', '价', '任', '份', '仿', '伍', '伏', '休', '众', '优', '伙', '会', '伟', '传', '伤', '伯', '估', '伴', '伸', '似', '但', '位', '低', '住', '体', '何', '余', '佛', '作', '你', '佩', '使', '依', '侧', '侯', '便', '俊', '俗', '保', '信', '修', '倍', '倒', '候', '借', '倩', '值', '倾', '假', '偏', '做', '停', '健', '偷', '傅', '储', '催', '傲', '傻', '像', '儿', '元', '兄', '充', '先', '光', '克', '免', '入', '全', '八', '公', '六', '兮', '兰', '共', '关', '兴', '兵', '其', '具', '典', '养', '兽', '内', '再', '冒', '写', '军', '冥', '冬', '冯', '冰', '冲', '决', '况', '冷', '净', '准', '凉', '凌', '减', '凑', '凝', '几', '凡', '凤', '凭', '凯', '凰', '凶', '出', '击', '刀', '分', '切',

In [18]:
t1 = time.time()
seqLen = 45
step = 1
sequences = []
nextWords = []
seqIgnored = 0
for i in range(0, len(allTokens) - seqLen, step):
    if len(set(allTokens[i:i+seqLen+1]).intersection(skipWords)) == 0:
        sequences.append(allTokens[i:i + seqLen])
        nextWords.append(allTokens[i + seqLen])
    else:
        seqIgnored += 1

print("Number of sequences ignored: ", seqIgnored)
print("Number of remaining sequences: ", len(sequences))

t2 = time.time()
print("Runtime for this cell in seconds: ", t2 - t1)

Number of sequences ignored:  21374058
Number of remaining sequences:  2174305
Runtime for this cell in seconds:  80.92613506317139


In [19]:
# def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=20):
#     # shuffle at unison
#     print('Shuffling sentences')
#
#     tmp_sentences = []
#     tmp_next_word = []
#
#     for i in np.random.permutation(len(sentences_original)):
#         tmp_sentences.append(sentences_original[i])
#         tmp_next_word.append(next_original[i])
#
#     cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
#     x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
#     y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]
#
#     print("Size of training set = %d" % len(x_train))
#     print("Size of test set = %d" % len(y_test))
#     return (x_train, y_train), (x_test, y_test)



sentences, testSentences, nextWordsTrain, testNextWords = train_test_split(sequences, nextWords, test_size=0.2, shuffle=True)
print("Shuffled and split sentences")
print("Size of training set = " + str(len(sentences)))
print("Size of test set = " + str(len(testSentences)))

tr = 0
te = 0
for word in words:
    if word not in nextWordsTrain:
        print(word)
    elif word not in testNextWords:
        print(word)



Shuffled and split sentences
Size of training set = 1739444
Size of test set = 434861


tr:  1699
te:  1699


In [20]:
print(testSentences[:10])
print(testNextWords[:10])

sentences_train = "sentences_train.txt"
nextWords_train = "nextWords_train.txt"
sentences_tests = "sentences_tests.txt"
nextWords_tests = "nextWords_tests.txt"

with open(sentences_train, "w", encoding='gbk') as sentences_file:
    for sentence in sentences:
        tmp_sentence = ",".join(sentence)
        sentences_file.write(f"{tmp_sentence}\n")

with open(sentences_tests, "w", encoding='gbk') as sentences_tests_file:
    for sentence in testSentences:
        tmp_sentence = ",".join(sentence)
        sentences_tests_file.write(f"{tmp_sentence}\n")

with open(nextWords_train, "w", encoding='gbk') as nextWords_testFile:
    for nextword in nextWordsTrain:
        nextWords_testFile.write(f"{nextword}\n")

with open(nextWords_tests, "w", encoding='gbk') as nextWords_testFile:
    for nextword in testNextWords:
        nextWords_testFile.write(f"{nextword}\n")

[['势', '在', '不', '断', '的', '增', '高', '这', '一', '发', '现', '让', '叶', '晨', '峰', '心', '里', '面', '有', '底', '了', '这', '也', '就', '证', '明', '叶', '晨', '峰', '他', '们', '在', '往', '高', '处', '奔', '跑', '只', '要', '奔', '跑', '到', '这', '条', '道'], ['议', '让', '雷', '正', '阳', '参', '加', '这', '一', '次', '的', '行', '动', '因', '为', '他', '一', '刀', '斩', '杀', '兽', '化', '的', '冷', '风', '就', '已', '经', '展', '现', '出', '强', '大', '的', '实', '力', '现', '在', '正', '是', '需', '要', '帮', '手', '的'], ['白', '琴', '能', '不', '惊', '讶', '吗', '这', '可', '是', '她', '一', '直', '躲', '避', '甚', '至', '想', '都', '不', '敢', '想', '的', '事', '现', '在', '却', '要', '去', '面', '对', '白', '琴', '还', '没', '有', '做', '好', '这', '个', '准', '备', '呢', '也', '许'], ['让', '我', '去', '核', '查', '资', '料', '开', '玩', '笑', '小', '学', '毕', '业', '后', '我', '就', '很', '少', '用', '脑', '子', '了', '石', '林', '笑', '着', '说', '道', '既', '然', '张', '舒', '君', '想', '利', '用', '他', '跟', '他', '玩', '阴', '的', '那', '石'], ['在', '法', '则', '感', '悟', '上', '赶', '上', '渡', '劫', '期', '修', '士', '要', '多', '花', '费', '不', 